In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

     |████████████████████████████████| 194kB 2.8MB/s 
     |████████████████████████████████| 51kB 3.2MB/s 


# Tabular training

> How to use the tabular application in fastai

To illustrate the tabular application, we will use the example of the [Adult dataset](https://archive.ics.uci.edu/ml/datasets/Adult) where we have to predict if a person is earning more or less than $50k per year using some general data.

In [ ]:
from fastai.tabular.all import *

We can download a sample of this dataset with the usual `untar_data` command:

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('/root/.fastai/data/adult_sample/adult.csv'),Path('/root/.fastai/data/adult_sample/models'),Path('/root/.fastai/data/adult_sample/export.pkl')]

Then we can have a look at how the data is structured:

In [ ]:
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


Some of the columns are continuous (like age) and we will treat them as float numbers we can feed our model directly. Others are categorical (like workclass or education) and we will convert them to a unique index that we will feed to embedding layers. We can specify our categorical and continuous column names, as well as the name of the dependent variable in `TabularDataLoaders` factory methods:

In [ ]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [Categorify, FillMissing, Normalize])

The last part is the list of pre-processors we apply to our data:

- `Categorify` is going to take every categorical variable and make a map from integer to unique categories, then replace the values by the corresponding index.
- `FillMissing` will fill the missing values in the continuous variables by the median of existing values (you can choose a specific value if you prefer)
- `Normalize` will normalize the continuous variables (substract the mean and divide by the std)



To further expose what's going on below the surface, let's rewrite this utilizing `fastai`'s `TabularPandas` class. We will need to make one adjustment, which is defining how we want to split our data. By default the factory method above used a random 80/20 split, so we will do the same:

In [ ]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [ ]:
to = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits)

Before finally building our `DataLoaders` again:

In [ ]:
dls = to.dataloaders(bs=64)

> Later we will explore why using `TabularPandas` to preprocess will be valuable.

The `show_batch` method works like for every other application:

In [ ]:
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,HS-grad,Never-married,Other-service,Not-in-family,White,False,36.000000,129590.998970,9.0,<50k
1,State-gov,Some-college,Never-married,Other-service,Not-in-family,White,False,24.000000,324637.006060,10.0,<50k
2,Private,Some-college,Divorced,Other-service,Own-child,White,False,50.000000,172942.000731,10.0,<50k
3,State-gov,11th,Never-married,Adm-clerical,Own-child,White,False,17.999999,59341.998628,7.0,<50k
4,Private,HS-grad,Divorced,Craft-repair,Unmarried,White,False,35.000000,36988.995066,9.0,<50k
5,?,Some-college,Never-married,?,Unmarried,Black,False,31.000000,163890.000275,10.0,<50k
6,Private,Bachelors,Married-civ-spouse,Prof-specialty,Own-child,White,False,29.000000,203697.000564,13.0,<50k
7,Private,HS-grad,Widowed,Craft-repair,Unmarried,White,False,58.000000,275859.001405,9.0,>=50k
8,Private,Assoc-acdm,Married-civ-spouse,Prof-specialty,Husband,White,False,42.000000,45363.005653,12.0,<50k
9,Private,HS-grad,Never-married,Other-service,Own-child,White,False,27.000000,150296.000452,9.0,<50k


We can define a model using the `tabular_learner` method. When we define our model, `fastai` will try to infer the loss function based on our `y_names` earlier. 

**Note**: Sometimes with tabular data, your `y`'s may be encoded (such as 0 and 1). In such a case you should explicitly pass `y_block = CategoryBlock` in your constructor so `fastai` won't presume you are doing regression.

In [ ]:
learn = tabular_learner(dls, metrics=accuracy)

And we can train that model with the `fit_one_cycle` method (the `fine_tune` method won't be useful here since we don't have a pretrained model).

In [ ]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.359080,0.344087,0.842905,00:06


We can then have a look at some predictions:

In [ ]:
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,1.0,2.0,1.0,1.0,4.0,5.0,1.0,-0.340913,1.483969,-1.208072,0.0,0.0
1,5.0,12.0,1.0,2.0,5.0,5.0,1.0,1.271127,-0.397830,-0.423943,0.0,0.0
2,5.0,2.0,3.0,8.0,1.0,5.0,1.0,1.490951,-0.863694,-1.208072,0.0,0.0
3,5.0,16.0,5.0,6.0,2.0,5.0,1.0,0.245284,-0.053319,-0.031878,0.0,0.0
4,5.0,16.0,5.0,9.0,4.0,3.0,1.0,0.025460,-0.322694,-0.031878,0.0,0.0
5,5.0,10.0,3.0,11.0,1.0,5.0,1.0,0.245284,-1.516208,1.144316,0.0,1.0
6,8.0,16.0,4.0,2.0,5.0,2.0,1.0,-0.340913,-1.603753,-0.031878,0.0,0.0
7,5.0,12.0,6.0,4.0,4.0,5.0,1.0,-1.000384,0.446380,-0.423943,0.0,0.0
8,7.0,11.0,3.0,11.0,1.0,5.0,1.0,0.831480,-0.208099,2.320510,1.0,1.0


Or use the predict method on a row:

In [ ]:
row, clas, probs = learn.predict(df.iloc[0])

In [ ]:
row.show()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Assoc-acdm,Married-civ-spouse,#na#,Wife,White,False,49.0,101320.002202,12.0,<50k


In [ ]:
clas, probs

(tensor(0), tensor([0.5973, 0.4027]))

To get prediction on a new dataframe, you can use the `test_dl` method of the `DataLoaders`. That dataframe does not need to have the dependent variable in its column.

In [ ]:
test_df = df.copy()
test_df.drop(['salary'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

Then `Learner.get_preds` will give you the predictions:

In [ ]:
learn.get_preds(dl=dl)

(tensor([[0.5973, 0.4027],
         [0.5205, 0.4795],
         [0.9780, 0.0220],
         ...,
         [0.6152, 0.3848],
         [0.7443, 0.2557],
         [0.6339, 0.3661]]), None)

## `fastai` with Other Libraries

As mentioned earlier, `TabularPandas` is a powerful and easy preprocessing tool for tabular data. Integration with libraries such as Random Forests and XGBoost requires only one extra step, that the `.dataloaders` call did for us. Let's look at our `to` again. It's values are stored in a `DataFrame` like object, where we can extract the `cats`, `conts,` `xs` and `ys` if we want to:

In [ ]:
to.xs[:3]

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
17304,5,12,3,4,1,5,1,-0.340913,-1.166621,-0.423943
8926,5,1,3,8,1,5,1,-1.293482,0.669804,-1.600137
13503,6,12,3,11,1,3,1,1.344402,-1.208433,-0.423943


To then preprocess our data, all we need to do is call `process` to apply all of our `procs` inplace:

In [ ]:
to.process()
to.xs[:3]

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
17304,0,0,0,0,0,0,1,-2.857227,-1.793325,-4.118739
8926,0,0,0,0,0,0,1,-2.927026,-1.793308,-4.579883
13503,0,0,0,0,0,0,1,-2.733737,-1.793326,-4.118739


Now that everything is encoded, you can then send this off to XGBoost or Random Forests by extracting the train and validation sets and their values:

In [ ]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

And now we can directly send this in!